In [71]:
import json
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix


file = '../data_resources/topics/vng_training.json'
data = json.load(open(file))

In [72]:
# transformer = TfidfVectorizer(smooth_idf=False)|
transformer = CountVectorizer()

corpus = []
y = []

for obj in data:
    content = obj['content']
    corpus.append(content)
    
    y.append(obj['sub_topic'])

In [73]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42)


X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

In [74]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Classifiers experimented with and score achieved:
classifiers = {
#     'knn': KNeighborsClassifier(3), # 0.18
    'svm1': SVC(kernel="linear", C=1.75), # 0.66
#     'svm2': SVC(gamma=2, C=1), # Took a long time - 0.54
#     'tree': DecisionTreeClassifier(max_depth=75), # 0.43
    'randomf': RandomForestClassifier(max_depth=20,n_estimators=30,max_features='auto',n_jobs=-1), # 0.61
}

In [68]:
for clf in classifiers:
    print(clf)
    classifier = classifiers[clf]

    %time clf = OneVsRestClassifier(classifier).fit(X_train, y_train)
    %time y_pred = clf.predict(X_test)

    score = accuracy_score(y_test, y_pred)
    conf = confusion_matrix(y_test, y_pred)
   
    print(score)

svm1
CPU times: user 1min 46s, sys: 22.9 ms, total: 1min 46s
Wall time: 1min 46s
CPU times: user 25.2 s, sys: 7.78 ms, total: 25.3 s
Wall time: 25.3 s
0.6666666666666666


In [ ]:
%time clf = OneVsRestClassifier(classifiers['randomf']).fit(X_train, y_train)

y_pred = clf.predict(X_test)

score = accuracy_score(y_test, y_pred)
print(score)